In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [15]:
## change ##
batch_size = 16
clip = .1
block = ResidualBlockGLU
normalFun = normalize_log2
#normalFun = normalize_log
name = 'Spec_2d_cnn_shareWeight'

In [3]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [4]:
yval=np.load('../Model/'+'gbm0'+'_val.npy',)
ytrain=np.load('../Model/'+'gbm0'+'_train.npy',)

Prepare data

In [5]:
train_gen = SequenceGenSpecBoost(train,0,int(500000000/150000),normalFun=normalFun,yhat=ytrain)
train_gen = DataLoader(train_gen,batch_size,False)
val_gen = SequenceGenSpecBoost(train,500000000,int((train.shape[0] - 500000000)/150000)-1,
                               normalFun=normalFun,yhat=yval)
val_gen = DataLoader(val_gen,batch_size,False)

Build model

In [36]:
model = Sequential(Dropout(0.5),
                      block(129),
                      Dropout(0.5),
                      block(258),
                      Dropout(0.5),
                      #block(516),
                      #Dropout(0.5),
                      #block(1032),
                      #Dropout(0.5),
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Linear(516,1,bias=False),
                      #LeakyReLU(0.2,True),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                      #biasLayer((1,)),
                      #Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [37]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [38]:
loss_func_nor = loss_func_generator_xgb(L1Loss())

Training

In [39]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:1.343814773628345, val_loss:2.3628346920013428
epoch:1, train_loss:1.3416035146667407, val_loss:2.3676400184631348
epoch:2, train_loss:1.3413263312182748, val_loss:2.3727917671203613
epoch:3, train_loss:1.3417794272446861, val_loss:2.374830484390259
epoch:4, train_loss:1.3407819429412484, val_loss:2.3753116130828857
Training completed in 53.310729026794434s
